### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [ ]:
# poetry add python-dotenv langchain langchain-openai

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_X


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    #model="openai/gpt-oss-120b",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 인터넷의 방대한 텍스트에서 다음 단어를 맞히는 방식으로 스스로 패턴을 학습합니다.  \n'
 '학습 중에는 사람이 정답을 주지 않고, 모델이 문맥을 활용해 스스로 단어의 관계를 파악합니다.  \n'
 '이렇게 얻은 지식을 바탕으로, 새로운 질문에 대해 자연스럽고 유착한 응답을 생성합니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [5]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
#combined_prompt.format(model_name="ChatGPT", count=3, language="한국어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"한국어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('ChatGPT 모델의 학습 원리는 다음과 같습니다.\n'
 '\n'
 'ChatGPT는 대규모 언어 모델로, 수십억 개의 매개변수를 가지고 있으며, 이를 통해 자연어 처리 작업을 수행합니다. 이 모델은 '
 '대규모의 텍스트 데이터를 학습하여 언어의 패턴과 구조를 이해하며, 이를 바탕으로 새로운 텍스트를 생성하거나 질문에 답변할 수 있습니다. '
 'ChatGPT의 학습에는 감독 학습과 강화 학습이 사용되며, 이를 통해 모델은 사용자 입력에 대한 적절한 응답을 생성하는 능력을 '
 '배웁니다.\n'
 '\n'
 'ChatGPT 모델의 장점은 다음과 같습니다.\n'
 '\n'
 '*   자연어 이해 능력: ChatGPT는 자연어 처리에 특화되어 있어, 사용자가 입력한 자연어를 이해하고 이에 적절한 응답을 제공할 수 '
 '있습니다.\n'
 '*   지식 정보 제공: ChatGPT는 방대한 양의 데이터를 학습했기 때문에, 사용자에게 다양한 주제에 대한 지식 정보를 제공할 수 '
 '있습니다.\n'
 '*   대화 관리: ChatGPT는 대화의 흐름을 관리하고, 상황에 맞는 응답을 제공할 수 있습니다.\n'
 '\n'
 'ChatGPT 모델과 비슷한 AI 모델은 다음과 같습니다.\n'
 '\n'
 '*   **메타 리자**: 메타 리자는 메타에서 개발한 대규모 언어 모델입니다. ChatGPT와 마찬가지로 자연어 이해 및 생성 능력에 '
 '중점을 두고 있습니다.\n'
 '*   **구글 바드**: 구글 바드

#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [7]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Gemma", "count": 4},
    {"model_name": "claude", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT-4는 방대한 텍스트 데이터를 통해 다음 단어를 예측하는 ‘예측 학습’ 방식으로 훈련됩니다.  \n'
 '학습 과정에서 인공 신경망이 문맥을 파악해 단어 간 확률 분포를 조정하며, 인간의 피드백(RLHF)으로 답의 품질을 미세 '
 '조정합니다.  \n'
 '결국 통계적 패턴 기반 생성이지만, 대규모 데이터와 강화학습이 결합되어 인간 수준의 자연스러운 대화 능력을 갖추게 됩니다.')
<class 'str'> Gemma 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemma는 Google의 Transformer 아키텍처를 기반으로 한 언어 모델로, 대규모 텍스트 데이터에서 다음 토큰을 예측하는 '
 '방식으로 학습됩니다.  \n'
 '모델은 인코더-디코더 구조를 사용하며, 주어진 입력 문맥에서 가장 적절한 단어를 예측하도록 최적화됩니다.  \n'
 '학습 과정에서는 마스크드 언어 모델링(MLM)과 같은 자기 지도 학습 기법을 활용하여 패턴을 스스로 학습합니다.  \n'
 '이를 통해 Gemma는 다양한 자연어 처리 작업에 대해 사전 학습된 지식을 활용할 수 있습니다.')
<class 'str'> claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Claude는 대규모 언어 모델로, 방대한 텍스트 데이터에서 다음 토큰을 예측하도록 학습됩니다.  \n'
 '인간 피드백 강화학습(RLHF)을 통해 유해한 답변은 줄이고 도움이 되는 답변은 강화합니다.  \n'
 '지식은 학습 시점에서 고정되며, 실시간 인터넷 검색은 하지 않습니다.  \n'
 '사용자와의 대화 맥락을 바탕으로 적절한 응답을 생성합니다.')


##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [8]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
ChatGPT는 “GPT(Generative Pre-trained Transformer)”라는 대규모 언어 모델을 기반으로 만들어졌습니다.  
GPT가 “말을 잘하게” 되는 과정은 크게 3단계로 나눌 수 있으며, 각 단계의 목표와 학습 방식이 다릅니다.

1. Pre-training(사전 학습) – “많은 글을 읽고 통계적 패턴을 기억”  
   • 목표: 다음 토큰(≈단어·부분 단어)을 맞추는 ‘자기 회귀’ 문제를 푸는 것.  
     예) “The weather is nice ___” → “today”일 확률이 높다는 사실을 학습.  
   • 데이터: 웹, 책, 위키, 논문 등 방대한 무표정 텍스트.  
   • 모델 구조:  
     – Transformer 디코더(마스크드 셀프 어텐션)  
     – 96층, 96-head, 임베딩 차원 12,288(d=12,288) 등 초거대 파라미터(175B).  
   • 최적화:  
     – Cross-entropy 손실 ↓, Adam, 1×10⁻⁴ lr, cosine 스케줄, bfloat16 혼합 정밀도.  
     – 수청만 달러 GPU 시간이 소요.  
   • 결과:  
     – ‘확률 시뮬레이터’가 됨. 주어진 앞 문맥 뒤에 올 토큰의 분포 pθ(tk|t<k)를 출력.  
     – 질문-답, 번역, 요약 등 다운스트림 임

In [ ]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [9]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [10]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍이 정말 좋아.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [19]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계에는 8개의 행성이 있습니다. 

1.  수성: 태양과 가장 가까운 행성으로, 표면이 암석으로 구성되어 있고 극도로 높은 온도와 낮은 온도가 반복됩니다.
2.  금성: 태양계에서 두 번째로 가까운 행성으로, 두꺼운 대기로 인해 극심한 온실 효과가 발생하여 매우 뜨겁습니다.
3.  지구: 우리가 사는 행성으로, 물과 대기가 있어 생명체가 존재할 수 있습니다.
4.  화성: 태양계에서 네 번째로 가까운 행성으로, 붉은색의 모래사막으로 덮여 있고 물과 생명체의 존재 가능성이 있습니다.
5.  목성: 태양계에서 가장 큰 행성으로, 가스 거인이며 강력한 자기장과 수많은 위성을 가지고 있습니다.
6.  토성: 태양계에서 두 번째로 큰 행성으로, 가스 거인이며 아름다운 고리를 가지고 있습니다.
7.  천왕성: 태양계에서 일곱 번째로 가까운 행성으로, 가스 거인이며 자전축이 기울어져 있어 극단적인 기후 변화를 경험합니다.
8.  해왕성: 태양계에서 가장 먼 행성으로, 가스 거인이며 강한 바람과 극적인 기후 변화를 가지고 있습니다.

이러한 행성들은 각각 고유한 특징과 성질을 가지고 있으며, 태양계에서 중요한 역할을 합니다.


##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [12]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input": "양자컴퓨터를 간략히 정리해 주세요."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [13]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 좋은 질문이에요.  
**태풍 발생**은 **여름~초가을(7~9월)**에 집중되긴 하지만, **가을에만 일어나는 대표적인 지구과학 현상**이라고 보기는 어렵습니다. 태풍은 계절보다는 **해수면 온도가 높은 시기**에 주로 발생하기 때문이죠.

---

### ✅ 가을에 주로 발생하는 **대표적인 지구과학 현상 3가지**:

1. **시차 현상 (Equinox)** – **추분(秋分)**  
   - 해가 **정활히 동쪽에서 떠서 정확히 서쪽으로 진다**는 현상  
   - 낮과 밤의 길이가 **거의 같아지는 시기** (약 9월 23일경)

2. **단풍 시작** – **기온 하강에 따른 식물의 생리적 반응**  
   - **엽록소 분해 → 안토시아닌 합성** 등으로 인해 **잎이 빨간색·노란색으로 변색**  
   - **기온이 낮고 일교차가 큰** 가을 날씨가 단풍에 **결정적 역할**을 함

3. **고기압의 이동** – **시베리아 고기압의 확장**  
   - **시베리아 고기압**이 남하하면서 **한반도에 차가운 건조한 공기**를 가져옴  
   - 이로 인해 **기온이 급강하**하고 **가을비 → 맑은 날씨**로 전환되는 **가을 전선** 형성

---

### 요약
태풍은 **여름~초가을**에 걸쳐 발생하지만, **가을에만 특별히 일어나는 현상**은 아닙니다.  
가을에 **지구과학적으로 특별한 현상**은 **추분, 단풍, 시베리아 고기압의 확장**이 훨씬 더 대표적이에요.


In [18]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season_name = get_current_season("north")  # 계절 값 얻기
print(f"현재 계절: {season_name}")


현재 계절: 가을


In [19]:


# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season_name}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.0
)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season_name": lambda x : season_name}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season_name}에 발생하는 자연 현상:\n{response}")


 가을에 발생하는 자연 현상:
1.  **추분과 추명** *   **날짜**: 매년 9월 23일, 추분은 낮과 밤의 길이가 같아지는 날 (평시 12시간) *   **설명은**: 추분은 태양이 적도 위를 지나가는 날로, 이 때 태평양 및 대서양에 있는 위성 천정각이 0도가 된다고 알려져 있습니다. (날씨가 추워지는 절기) *   **예상되는 영향**: 낮과 밤의 시간 변화에 따른 환경 조건 차이 때문일 수도 있지만, 식물에 대한 직접적인 영향은 적으로 추분에 의해 기상 재해 등 일교차가 크게 발생하지 말아야 할 듯 합니다. 

<!---->

1.  **성층권 구름(스트라토스페릭 클라우드)** *   **날짜**: 가을에 추운 날씨가 시작되면서, 대기의 습도가 높아지는 때 *   **설명은**: 성층권 구름은 높은 고도에서 나타나는 구름입니다. *   **예상되는 영향**: 스트라토스페릭 클라우드라고 알려진 구름들은 대기의 열대권 대기 온도를 낮추는 효과가 있는 것으로 추정되고 있습니다. 

<!---->

1.  **북극 저기압** *   **날짜**: 가을에 북극의 기온이 떨어지면서 약해지는 시점 *   **설명은**: 북극 저기압은 대기를 유지하는 온대적인 구조로, 약해지면 극지방의 고기압이 강해져 기상과 기후에 영향을 줍니다. *   **예상되는 영향**: 북극 저기압의 약화는 북극 지방에서 기상 패턴을 바꾸어 극한의 날씨를 나타낼 수 있습니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [20]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1377.98원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: ## 최근 환율 동향

2024년 3월 말, 미국 달러화 대비 한국 원화 환율은 1달러 = 1,377.98원입니다. 최근 몇 개월 동안 한국 원화 가치는 다양한 요인으로 인해 변동하고 있습니다.

## 경제 지표

* 미국의 금리 인상: 미국의 연방준비제도(Fed)는 인플레이션 억제를 위해 금리를 인상하고 있습니다. 이는 달러화 가치를 상승시키고 있습니다.
* 한국의 무역 수지: 한국의 무역 수지는 최근 수출 감소와 수입 증가로 인해 적자를 기록하고 있습니다. 이는 원화 가치를 하락시키고 있습니다.
* 한국의 금리 인상: 한국은행은 인플레이션 억제를 위해 금리를 인상하고 있습니다. 이는 원화 가치를 상승시키고 있습니다.

## 시장 전망

* 달러화 가치: 미국 경제가 여전히 견조한 성장세를 보이고 있고, 인플레이션이 높은 수준을 유지하고 있기 때문에 달러화 가치는 당분간 높은 수준을 유지할 것으로 전망됩니다.
* 원화 가치: 한국의 경제 성장세가 둔화되고 있고, 무역 수지 적자가 지속되고 있기 때문에 원화 가치는 당분간 약세를 보일 것으로 전망됩니다.

## 환율 전망

* 단기 전망: 달러화 가치가 높은 수준을 유지할 것으로 예상되기 때문에 원화 가치는 약세를 보일 것으로 전망됩니다. 따라서 환율은 1,380원 이상으로 상승할 가능성이 있습니다.
* 장기 전망: 한국의 경제가 회복되고, 무역 수지가 개선되면 원화 가치는 상승할 것으로 전망됩니다. 따라서 환율은 1,300원 이하로 하락할 가능성이 있습니다.

## 투자 전략

* 수출 기업: 달러화 가치가 상승하면 수출 기업의 수익성이 개선될 수 있습니다. 따라서 수출 기업은 달러화 가치 상승에 대비하여 환 헷지를 고려할 수 있습니다.
* 수입 기업: 달러화 가치가 상승하면 수입 기업의 비용이 증가할 수 있습니다. 따라서 수입 기업은 달러화 가치 상승에 대비하여 환 헷지를 고려할 